In [1]:
import polars as pl
import matplotlib.pyplot as plt

from common.constants.column_types import CPZP_SCHEMA, OZP_SCHEMA
from common.constants.column_names import SHARED_COLUMNS, OZP_COLUMNS

pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(100)

polars.config.Config

In [2]:
FILE_PATH = "./DATACON_data/OZP_preskladane.csv"

In [3]:
def read_preskladane_data(file_path: str, schema: pl.Schema) -> pl.DataFrame:
    return pl.read_csv(
        file_path,
        null_values=["NA", ""],
        schema=schema,
    )

In [4]:
ozp_df = read_preskladane_data(FILE_PATH, OZP_SCHEMA)

print("OZP DATA: ", ozp_df.schema)

OZP DATA:  Schema({'Id_pojistence': String, 'Pohlavi': Enum(categories=['M', 'F']), 'Rok_narozeni': Int64, 'Posledni_zahajeni_pojisteni': Date, 'Posledni_ukonceni_pojisteni': Date, 'Datum_umrti': Date, 'Typ_udalosti': Enum(categories=['předpis', 'vakcinace']), 'Detail_udalosti': Int64, 'Nazev': String, 'Pocet_baleni': Float64, 'Datum_udalosti': Date, 'léková_forma_zkr': String, 'ATC_skupina': String, 'síla': String, 'doplněk_názvu': String, 'léková_forma': String, 'léčivé_látky': String, 'Equiv_sloucenina': Enum(categories=['TRIAMCINOLON', 'FLUDROKORTISON', 'METHYLPREDNISOLON', 'DEXAMETHASON', 'PREDNISON', 'BETAMETHASON', 'HYDROKORTISON']), 'Prednison_equiv': Float64, 'Pocet_v_baleni': Float64, 'pocet_vakcinaci': Int64, 'ockovany': Enum(categories=['1', '0']), 'pocet_predpisu': Int64})


In [5]:
ozp_alive_df = ozp_df.filter(pl.col(SHARED_COLUMNS.DATUM_UMRTI).is_null())

print(ozp_alive_df)

shape: (3_421_491, 23)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ Id_ ┆ Poh ┆ Rok ┆ Pos ┆ Pos ┆ Dat ┆ Typ ┆ Det ┆ Naz ┆ Poc ┆ Dat ┆ lék ┆ ATC ┆ síl ┆ dop ┆ lék ┆ léč ┆ Equ ┆ Pre ┆ Poc ┆ poc ┆ ock ┆ poc │
│ poj ┆ lav ┆ _na ┆ led ┆ led ┆ um_ ┆ _ud ┆ ail ┆ ev  ┆ et_ ┆ um_ ┆ ová ┆ _sk ┆ a   ┆ lně ┆ ová ┆ ivé ┆ iv_ ┆ dni ┆ et_ ┆ et_ ┆ ova ┆ et_ │
│ ist ┆ i   ┆ roz ┆ ni_ ┆ ni_ ┆ umr ┆ alo ┆ _ud ┆ --- ┆ bal ┆ uda ┆ _fo ┆ upi ┆ --- ┆ k_n ┆ _fo ┆ _lá ┆ slo ┆ son ┆ v_b ┆ vak ┆ ny  ┆ pre │
│ enc ┆ --- ┆ eni ┆ zah ┆ uko ┆ ti  ┆ sti ┆ alo ┆ str ┆ eni ┆ los ┆ rma ┆ na  ┆ str ┆ ázv ┆ rma ┆ tky ┆ uce ┆ _eq ┆ ale ┆ cin ┆ --- ┆ dpi │
│ e   ┆ enu ┆ --- ┆ aje ┆ nce ┆ --- ┆ --- ┆ sti ┆     ┆ --- ┆ ti  ┆ _zk ┆ --- ┆     ┆ u   ┆ --- ┆ --- ┆ nin ┆ uiv ┆ ni  ┆ aci ┆ enu ┆ su  │
│ --- ┆ m   ┆ i64 ┆ ni_ ┆ ni_ ┆ dat ┆ enu ┆ --- ┆     ┆ f64 ┆ --- ┆ r   ┆ str ┆     ┆ --- ┆ str ┆ str ┆ a   ┆ --- ┆ --- ┆ --- ┆ m   ┆ ---

In [6]:
def get_vekova_kohorta(vek):
    if vek >= 80:
        return "80+"
    elif vek >= 70:
        return "70-79"
    elif vek >= 65:
        return "65-69"
    elif vek >= 60:
        return "60-64"
    elif vek >= 50:
        return "50-59"
    elif vek >= 30:
        return "30-49"
    elif vek >= 12:
        return "12-29"
    else:
        return "<12"


ocko_df = (
    ozp_alive_df.filter(pl.col("Typ_udalosti") == "vakcinace")
    .sort(["Id_pojistence", "Datum_udalosti"])
    .with_columns(pl.arange(0, pl.len()).over("Id_pojistence").alias("Cislo_davky") + 1)
    .with_columns(
        (pl.col("Datum_udalosti").dt.year() - pl.col("Rok_narozeni")).alias(
            "Vek_v_dobe_vakcinace"
        )
    )
    .with_columns(
        pl.col("Vek_v_dobe_vakcinace")
        .map_elements(get_vekova_kohorta, return_dtype=pl.Utf8)
        .alias("Vekova_kohorta")
    )
    .select(
        [
            "Id_pojistence",
            pl.col("Datum_udalosti").alias("Datum_vakcinace"),
            "Cislo_davky",
            "Vekova_kohorta",
        ]
    )
)


predpis_df = (
    ozp_alive_df.filter(pl.col("Typ_udalosti") == "předpis")
    .sort(["Id_pojistence", "Datum_udalosti"])
    .with_columns(
        pl.arange(0, pl.len()).over("Id_pojistence").alias("cislo_predpisu") + 1
    )
    .with_columns(
        (pl.col("Datum_udalosti").dt.year() - pl.col("Rok_narozeni")).alias(
            "Vek_v_dobe_predpisu"
        )
    )
    .with_columns(
        pl.col("Vek_v_dobe_predpisu")
        .map_elements(get_vekova_kohorta, return_dtype=pl.Utf8)
        .alias("Vekova_kohorta")
    )
    .select(
        [
            "Id_pojistence",
            pl.col("Datum_udalosti").alias("Datum_predpisu"),
            "cislo_predpisu",
            "Vekova_kohorta",
            "Prednison_equiv",
        ]
    )
)

print(ocko_df)
print(predpis_df)

predpis_with_ocko_df = (
    predpis_df.join_asof(
        ocko_df,
        left_on="Datum_predpisu",
        right_on="Datum_vakcinace",
        by="Id_pojistence",
        strategy="backward",
    )
    .with_columns(
        [pl.col("Cislo_davky").fill_null(0).alias("Pocet_davek_v_dobe_predpisu")]
    )
    .select(
        [
            "Id_pojistence",
            "Datum_predpisu",
            "cislo_predpisu",
            "Vekova_kohorta",
            "Pocet_davek_v_dobe_predpisu",
            "Prednison_equiv",
        ]
    )
    .filter(pl.col("Prednison_equiv").is_not_null())
)

print(predpis_with_ocko_df)

shape: (1_311_187, 4)
┌───────────────┬─────────────────┬─────────────┬────────────────┐
│ Id_pojistence ┆ Datum_vakcinace ┆ Cislo_davky ┆ Vekova_kohorta │
│ ---           ┆ ---             ┆ ---         ┆ ---            │
│ str           ┆ date            ┆ i64         ┆ str            │
╞═══════════════╪═════════════════╪═════════════╪════════════════╡
│ ANPOJ00000001 ┆ 2021-07-26      ┆ 1           ┆ 12-29          │
│ ANPOJ00000001 ┆ 2021-09-06      ┆ 2           ┆ 12-29          │
│ ANPOJ00000001 ┆ 2022-02-23      ┆ 3           ┆ 12-29          │
│ ANPOJ00000005 ┆ 2021-06-27      ┆ 1           ┆ 12-29          │
│ ANPOJ00000005 ┆ 2021-07-17      ┆ 2           ┆ 12-29          │
│ ANPOJ00000007 ┆ 2021-08-20      ┆ 1           ┆ 12-29          │
│ ANPOJ00000007 ┆ 2021-09-10      ┆ 2           ┆ 12-29          │
│ ANPOJ00000008 ┆ 2021-05-25      ┆ 1           ┆ 30-49          │
│ ANPOJ00000008 ┆ 2021-06-15      ┆ 2           ┆ 30-49          │
│ ANPOJ00000010 ┆ 2021-11-24      ┆ 1   

/tmp/ipykernel_59101/3461876727.py:76: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  predpis_df.join_asof(


In [7]:
# === JUST FOR TESTING IF THE JOIN IS CORRECT ===
#  print(predpis_with_ocko_df.filter(pl.col("Pocet_davek_v_dobe_predpisu") > 0))

# test_id = "ANPOJ00000060"
# print(ocko_df.filter(pl.col("Id_pojistence") == test_id))
# print(predpis_df.filter(pl.col("Id_pojistence") == test_id))

# print(predpis_with_ocko_df.filter(pl.col("Id_pojistence") == test_id))

In [8]:
vakcinace_grouped = (
    ocko_df.group_by(["Vekova_kohorta", "Datum_vakcinace", "Cislo_davky"])
    .agg(pl.len().alias("pocet_vakcinaci"))
    .sort(["Vekova_kohorta", "Datum_vakcinace", "Cislo_davky"])
)

print(vakcinace_grouped)

for vek_kohorta in vakcinace_grouped["Vekova_kohorta"].unique():
    for cislo_davky in vakcinace_grouped["Cislo_davky"].unique():
        data = vakcinace_grouped.filter(
            pl.col("Vekova_kohorta") == vek_kohorta,
            pl.col("Cislo_davky") == cislo_davky,
        )

        plt.figure(figsize=(10, 4))
        plt.plot(
            data["Datum_vakcinace"], data["pocet_vakcinaci"], marker="o", linestyle="-"
        )
        plt.title(
            f"Počet vakcinací: věková skupina {vek_kohorta}, při {cislo_davky}. dávce"
        )
        plt.xlabel("Datum vakcinace")
        plt.ylabel("Počet vakcinací")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"out/whole_period/cohort_{vek_kohorta}_{cislo_davky}.png")
        plt.close()

shape: (16_338, 4)
┌────────────────┬─────────────────┬─────────────┬─────────────────┐
│ Vekova_kohorta ┆ Datum_vakcinace ┆ Cislo_davky ┆ pocet_vakcinaci │
│ ---            ┆ ---             ┆ ---         ┆ ---             │
│ str            ┆ date            ┆ i64         ┆ u32             │
╞════════════════╪═════════════════╪═════════════╪═════════════════╡
│ 12-29          ┆ 2020-12-27      ┆ 1           ┆ 17              │
│ 12-29          ┆ 2020-12-28      ┆ 1           ┆ 26              │
│ 12-29          ┆ 2020-12-29      ┆ 1           ┆ 38              │
│ 12-29          ┆ 2020-12-30      ┆ 1           ┆ 25              │
│ 12-29          ┆ 2020-12-31      ┆ 1           ┆ 9               │
│ 12-29          ┆ 2021-01-02      ┆ 1           ┆ 15              │
│ 12-29          ┆ 2021-01-03      ┆ 1           ┆ 4               │
│ 12-29          ┆ 2021-01-04      ┆ 1           ┆ 25              │
│ 12-29          ┆ 2021-01-05      ┆ 1           ┆ 29              │
│ 12-29        

In [9]:
predpisy_grouped = (
    predpis_with_ocko_df.group_by(
        ["Vekova_kohorta", "Datum_predpisu", "Pocet_davek_v_dobe_predpisu"]
    )
    .agg(pl.len().alias("pocet_predpisu"))
    .sort(["Vekova_kohorta", "Datum_predpisu", "Pocet_davek_v_dobe_predpisu"])
)

# for vek_kohorta in predpisy_grouped["Vekova_kohorta"].unique():
#     for pocet_davky in predpisy_grouped["Pocet_davek_v_dobe_predpisu"].unique():
#         predpis_data = predpisy_grouped.filter(
#             pl.col("Vekova_kohorta") == vek_kohorta,
#             pl.col("Pocet_davek_v_dobe_predpisu") == pocet_davky,
#         )
#         vax_data = vakcinace_grouped.filter(
#             pl.col("Vekova_kohorta") == vek_kohorta,
#             pl.col("Cislo_davky") == cislo_davky,
#         )

#         plt.figure(figsize=(10, 4))
#         plt.plot(predpis_data["Datum_predpisu"], predpis_data["pocet_predpisu"], linestyle="-")
#         plt.title(
#             f"Počet predpisů: věková skupina {vek_kohorta}, při {pocet_davky}. dávce"
#         )
#         plt.xlabel("Datum predpisu")
#         plt.ylabel("Počet predpisů")
#         plt.xticks(rotation=45)
#         plt.grid(True)
#         plt.tight_layout()
#         plt.savefig(f"out/predpis_whole_period/cohort_{vek_kohorta}_{pocet_davky}.png")
#         plt.close()

for vek_kohorta in predpisy_grouped["Vekova_kohorta"].unique():
    for pocet_davky in predpisy_grouped["Pocet_davek_v_dobe_predpisu"].unique():
        predpis_data = predpisy_grouped.filter(
            pl.col("Vekova_kohorta") == vek_kohorta,
            pl.col("Pocet_davek_v_dobe_predpisu") == pocet_davky,
        )
        vax_data = vakcinace_grouped.filter(
            pl.col("Vekova_kohorta") == vek_kohorta,
            pl.col("Cislo_davky")
            == pocet_davky,  # corrected: cislo_davky → pocet_davky
        )

        if predpis_data.is_empty() or vax_data.is_empty():
            continue  # skip empty plots

        fig, ax1 = plt.subplots(figsize=(10, 4))

        # Plot prescriptions (left y-axis)
        ax1.plot(
            predpis_data["Datum_predpisu"],
            predpis_data["pocet_predpisu"],
            linestyle="-",
            label="Počet předpisů",
            color="tab:blue",
        )
        ax1.set_xlabel("Datum")
        ax1.set_ylabel("Počet předpisů", color="tab:blue")
        ax1.tick_params(axis="y", labelcolor="tab:blue")
        ax1.set_title(f"Počet předpisů a očkování: {vek_kohorta}, {pocet_davky}. dávka")
        ax1.tick_params(axis="x", rotation=45)
        ax1.grid(True)

        # Plot vaccinations (right y-axis)
        ax2 = ax1.twinx()
        ax2.plot(
            vax_data["Datum_vakcinace"],
            vax_data["pocet_vakcinaci"],
            linestyle="-",
            label="Počet očkování",
            color="tab:red",
        )
        ax2.set_ylabel("Počet očkování", color="tab:red")
        ax2.tick_params(axis="y", labelcolor="tab:red")

        fig.tight_layout()
        plt.savefig(f"out/predpis_whole_period/cohort_{vek_kohorta}_{pocet_davky}.png")

        plt.close()

In [10]:
prvopredpisy_grouped = (
    predpis_with_ocko_df.filter(pl.col("cislo_predpisu") == 1)
    .group_by(["Vekova_kohorta", "Datum_predpisu", "Pocet_davek_v_dobe_predpisu"])
    .agg(pl.len().alias("pocet_prvopredpisu"))
    .sort(["Vekova_kohorta", "Datum_predpisu", "Pocet_davek_v_dobe_predpisu"])
)

for vek_kohorta in prvopredpisy_grouped["Vekova_kohorta"].unique():
    for pocet_davky in prvopredpisy_grouped["Pocet_davek_v_dobe_predpisu"].unique():
        data = prvopredpisy_grouped.filter(
            pl.col("Vekova_kohorta") == vek_kohorta,
            pl.col("Pocet_davek_v_dobe_predpisu") == pocet_davky,
        )

        plt.figure(figsize=(10, 4))
        plt.plot(
            data["Datum_predpisu"],
            data["pocet_prvopredpisu"],
            linestyle="-",
        )
        plt.title(
            f"Počet prvopredpisů: věková skupina {vek_kohorta}, při {pocet_davky}. dávce"
        )
        plt.xlabel("Datum predpisu")
        plt.ylabel("Počet predpisů")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(
            f"out/prvopredpisy_whole_period/cohort_{vek_kohorta}_{pocet_davky}.png"
        )
        plt.close()

In [ ]:
prednison_grouped = (
    predpis_with_ocko_df.group_by(
        ["Vekova_kohorta", "Datum_predpisu", "Pocet_davek_v_dobe_predpisu"]
    )
    .agg(pl.sum("Prednison_equiv").alias("suma_prednison_equ"))
    .sort(["Vekova_kohorta", "Datum_predpisu", "Pocet_davek_v_dobe_predpisu"])
)

for vek_kohorta in prednison_grouped["Vekova_kohorta"].unique():
    for pocet_davky in prednison_grouped["Pocet_davek_v_dobe_predpisu"].unique():
        data = prednison_grouped.filter(
            pl.col("Vekova_kohorta") == vek_kohorta,
            pl.col("Pocet_davek_v_dobe_predpisu") == pocet_davky,
        )

        plt.figure(figsize=(10, 4))
        plt.plot(data["Datum_predpisu"], data["suma_prednison_equ"], linestyle="-")
        plt.title(
            f"Suma prednison_equ: věková skupina {vek_kohorta}, při {pocet_davky}. dávce"
        )
        plt.xlabel("Datum predpisu")
        plt.ylabel("Suma prednison_equ")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(
            f"out/prednison_equ_whole_period/cohort_{vek_kohorta}_{pocet_davky}.png"
        )
        plt.close()

shape: (5, 4)
┌───────────────┬─────────────────┬─────────────┬────────────────┐
│ Id_pojistence ┆ Datum_vakcinace ┆ Cislo_davky ┆ Vekova_kohorta │
│ ---           ┆ ---             ┆ ---         ┆ ---            │
│ str           ┆ date            ┆ i64         ┆ str            │
╞═══════════════╪═════════════════╪═════════════╪════════════════╡
│ ANPOJ00000001 ┆ 2021-07-26      ┆ 1           ┆ 12-29          │
│ ANPOJ00000001 ┆ 2021-09-06      ┆ 2           ┆ 12-29          │
│ ANPOJ00000001 ┆ 2022-02-23      ┆ 3           ┆ 12-29          │
│ ANPOJ00000005 ┆ 2021-06-27      ┆ 1           ┆ 12-29          │
│ ANPOJ00000005 ┆ 2021-07-17      ┆ 2           ┆ 12-29          │
└───────────────┴─────────────────┴─────────────┴────────────────┘
shape: (16_338, 4)
┌────────────────┬─────────────────┬─────────────┬─────────────────┐
│ Vekova_kohorta ┆ Datum_vakcinace ┆ Cislo_davky ┆ pocet_vakcinaci │
│ ---            ┆ ---             ┆ ---         ┆ ---             │
│ str            ┆ date

In [ ]:
rozhodna_data = (
    vakcinace_grouped.sort(
        ["Vekova_kohorta", "Cislo_davky", "pocet_vakcinaci", "Datum_vakcinace"],
        descending=[False, False, True, False],
    )
    .group_by(["Vekova_kohorta", "Cislo_davky"])
    .agg([pl.col("Datum_vakcinace").first().alias("Datum_rozhodne")])
)

# print(rozhodna_data)


filtered_ocko_df = (
    ocko_df.join(rozhodna_data, on=["Vekova_kohorta", "Cislo_davky"], how="left")
    .with_columns(
        (pl.col("Datum_vakcinace") - pl.col("Datum_rozhodne"))
        .dt.total_days()
        .alias("delta_dni")
    )
    .filter(pl.col("delta_dni").abs() <= 30)
)
print(filtered_ocko_df)


for vek_kohorta in filtered_ocko_df["Vekova_kohorta"].unique():
    for cislo_davky in filtered_ocko_df["Cislo_davky"].unique():
        data = filtered_ocko_df.filter(
            pl.col("Vekova_kohorta") == vek_kohorta,
            pl.col("Cislo_davky") == cislo_davky,
        )
        if data.shape[0] > 10:
            print(data)


# cohort_grouped_df = (
#     filtered_ocko_df.group_by(["Vekova_kohorta", "Datum_vakcinace", "Cislo_davky"])
#     .agg(pl.len().alias("pocet_vakcinaci"))
#     .sort(["Vekova_kohorta", "Datum_vakcinace", "Cislo_davky"])
# )

# print(cohort_grouped_df)

shape: (787_200, 6)
┌───────────────┬─────────────────┬─────────────┬────────────────┬────────────────┬───────────┐
│ Id_pojistence ┆ Datum_vakcinace ┆ Cislo_davky ┆ Vekova_kohorta ┆ Datum_rozhodne ┆ delta_dni │
│ ---           ┆ ---             ┆ ---         ┆ ---            ┆ ---            ┆ ---       │
│ str           ┆ date            ┆ i64         ┆ str            ┆ date           ┆ i64       │
╞═══════════════╪═════════════════╪═════════════╪════════════════╪════════════════╪═══════════╡
│ ANPOJ00000001 ┆ 2021-07-26      ┆ 1           ┆ 12-29          ┆ 2021-08-02     ┆ -7        │
│ ANPOJ00000001 ┆ 2021-09-06      ┆ 2           ┆ 12-29          ┆ 2021-08-23     ┆ 14        │
│ ANPOJ00000001 ┆ 2022-02-23      ┆ 3           ┆ 12-29          ┆ 2022-01-28     ┆ 26        │
│ ANPOJ00000007 ┆ 2021-08-20      ┆ 1           ┆ 12-29          ┆ 2021-08-02     ┆ 18        │
│ ANPOJ00000007 ┆ 2021-09-10      ┆ 2           ┆ 12-29          ┆ 2021-08-23     ┆ 18        │
│ ANPOJ00000008 ┆ 20